In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time, re
import requests
import json

In [2]:
main_page = 'http://catalogue.usc.edu/content.php?catoid=7&navoid=2081'
url  = requests.get(main_page)
data = url.text
soup = BeautifulSoup(data,"html.parser")

In [222]:
bachelor = ("Bachelor's Degree", [ (a.text,'catalogue.usc.edu/'+a['href']) for a in soup.find_all('ul')[2].find_all('a') ])

In [221]:
combinedmajor = ("Combined Major", [ (a.text,'catalogue.usc.edu/'+a['href']) for a in soup.find_all('ul')[3].find_all('a') ])

In [220]:
undecertificate = ("Undergraduate Certificate", [ (a.text,'catalogue.usc.edu/'+a['href']) for a in soup.find_all('ul')[4].find_all('a') ])

In [219]:
honor = ("Honors", [ (a.text,'catalogue.usc.edu/'+a['href']) for a in soup.find_all('ul')[5].find_all('a') ])

In [218]:
minor = ("Minor", [ (a.text,'catalogue.usc.edu/'+a['href']) for a in soup.find_all('ul')[6].find_all('a') ])

In [217]:
interminor = ("Interdisciplinary Minors", [ (a.text,'catalogue.usc.edu/'+a['href']) for a in soup.find_all('ul')[7].find_all('a') ])

In [216]:
jointdegree = ("Joint Degree", [ (a.text,'catalogue.usc.edu/'+a['href']) for a in soup.find_all('ul')[8].find_all('a') ])

In [215]:
diploma = ("Diploma", [ (a.text,'catalogue.usc.edu/'+a['href']) for a in soup.find_all('ul')[9].find_all('a') ])

In [214]:
mba = ("Master of Business Administration", [ (a.text,'catalogue.usc.edu/'+a['href']) for a in soup.find_all('ul')[10].find_all('a') ])

In [213]:
mbadual = ("MBA Dual Degree Programs", [ (a.text,'catalogue.usc.edu/'+a['href']) for a in soup.find_all('ul')[11].find_all('a') ])

In [212]:
master = ("Master's Degree", [ (a.text,'catalogue.usc.edu/'+a['href']) for a in soup.find_all('ul')[12].find_all('a') ])

In [211]:
univcertificate = ("University Certificate", [ (a.text,'catalogue.usc.edu/'+a['href']) for a in soup.find_all('ul')[13].find_all('a') ])

In [210]:
gradcertificate = ("Graduate Certificate", [ (a.text,'catalogue.usc.edu/'+a['href']) for a in soup.find_all('ul')[14].find_all('a') ])

In [209]:
dualdegree = ("Dual Degree", [ (a.text,'catalogue.usc.edu/'+a['href']) for a in soup.find_all('ul')[14].find_all('a') ])

In [208]:
docdegree = ("Doctoral Degree", [ (a.text,'catalogue.usc.edu/'+a['href']) for a in soup.find_all('ul')[15].find_all('a') ])

In [225]:
allprogram = [bachelor, combinedmajor, undecertificate, honor, minor, interminor, jointdegree, diploma, mba, mbadual, master, univcertificate, gradcertificate, dualdegree, docdegree]

In [206]:
def get_program(program,programtype,pid):
    pdict = {}
    programURL = 'http://'+program[1]
    programName = program[0]
    
    pdict['program_id'] = pid
    pdict['program_type'] = programtype
    pdict['program_url'] = programURL
    pdict['program_name'] = programName
    
    programPage = requests.get(programURL)
    soup = BeautifulSoup(programPage.text,"html.parser")
    courseLink = soup.findAll('li',{'class':'acalog-course'})
    href = [ i.find('a') for i in courseLink ]
    
    anchors = [ re.search('[0-9]{5}[0-9]?', i['onclick']).group() for i in href ]
    
    pdict['courses'] = []
    
    base1 = 'http://catalogue.usc.edu/ajax/preview_course.php?catoid=7&coid='
    base2 = '&display_options=a:2:{s:8:~location~;s:7:~program~;s:4:~core~;s:5:~34711~;}&show'
    for an in anchors:
        f = requests.get(base1+an+base2)
        content = BeautifulSoup(f.text, "html.parser").text
        content = content.replace('\r',' ').replace('\t',' ')
        content = content.replace(',',' ,').replace(':',' :').replace('.', ' .').replace(';', ' ;')
        code = re.search('[A-Z][A-Z]+ [0-9][0-9]+', content).group()
        
        title = content.split('\n')[1].split("  ")[-2]
        
        content = content.split("Units :")[-1].lstrip('0123456789.- ')
        content = content.split('Terms Offered : ')[-1]
        content = rm(content,'Fa')
        content = rm(content,'Sp')
        content = rm(content,'Sm')
        mcontent = content.split("  ")[0]
        
        desciption = mcontent.split('Prerequisite : ')[0]
        
        prerequisite = None
        preparation_text = ""
        preparation = None
        crosslist = None
        corequisite = None
        duplicate = None
        
        if 'Prerequisite : ' in mcontent:
            prerequisite = mcontent.split('Prerequisite : ')[-1]
            prerequisite = re.findall('[A-Z][A-Z]+ [0-9][0-9]+', prerequisite)
        
        note = content.split("  ")[1:]
        for n in note:
            if 'Recommended Preparation' in n:
                preparation = re.findall('[A-Z][A-Z]+ [0-9][0-9]+', n)
                preparation_text = n.split("Recommended Preparation :")[-1]
            elif 'Duplicates Credit' in n:
                duplicate = re.findall('[A-Z][A-Z]+ [0-9][0-9]+', n)
            elif 'Crosslisted' in n:
                crosslist = re.findall('[A-Z][A-Z]+ [0-9][0-9]+', n)
            elif 'Corequisite :' in n:
                corequisite = re.findall('[A-Z][A-Z]+ [0-9][0-9]+', n)
        
        cdict = {}
        cdict['course_id'] = code
        cdict['course_name'] = title
        cdict['description'] = desciption
        cdict['prerequisite'] = prerequisite
        cdict['preparation'] = preparation
        cdict['crosslist'] = crosslist
        cdict['corequisite'] = corequisite
        cdict['duplicate'] = duplicate
        
        pdict['courses'].append(cdict)
    return pdict

In [128]:
def rm(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text 

In [232]:
program_json = []
count = 0
for ptype in allprogram:
    programtype = ptype[0]
    for program in ptype[1]:
        print(program[0])
        program_json.append(get_program(program,programtype,'pgm'+format(count, '05d')))
        count += 1

Accounting (BS)
Acting for the Stage, Screen and New Media (BFA)
Aerospace Engineering (BS)
American Popular Culture
American Studies and Ethnicity (African American Studies) (BA)
American Studies and Ethnicity (Asian American Studies) (BA)
American Studies and Ethnicity (BA)
American Studies and Ethnicity (Chicano/Latino Studies) (BA)
Animation and Digital Arts (BA)
Anthropology (BA)
Anthropology (Visual Anthropology) (BA)
Applied and Computational Mathematics (BA)
Applied and Computational Mathematics (BS)
Applied Mechanics (BS)
Archaeology (BA)
Architectural Studies (BS)
Architecture (BArch)
Art (BA)
Art History (BA)
Arts, Technology and the Business of Innovation (BS)
Astronautical Engineering (BS)
Astronomy (BA)
Astronomy (BS)
Biochemistry (BS)
Biological Sciences (BA)
Biological Sciences (BS)
Biomedical Engineering (BS)
Biomedical Engineering, Biochemical Engineering Emphasis, (BS)
Biomedical Engineering, Electrical Engineering Emphasis, (BS)
Biomedical Engineering, Mechanical En

Environmental Chemistry and Sustainability Minor
Environmental Engineering Minor
Environmental Health Minor
Environmental Studies Minor
Ethics and Moral Philosophy Minor
Folklore and Popular Culture Minor
Forensics and Criminality Minor
French Minor
Future Cinema Minor
Game Animation Minor
Game Audio Minor
Game Design Minor
Game Entrepreneurism Minor
Game Studies Minor
Game User Research Minor
Gender and Social Justice Minor
Gender Studies Minor
Geobiology Minor
Geohazards Minor
German Minor
Global Communication Minor
Global Health Minor
Health Administration Minor
Health Care Studies Minor
Health Communication Minor
Health Policy Minor
Hip Hop, Street and Social Dance Forms Minor
History and Culture of Business Minor
History Minor
Human Movement Sciences Minor
Human Resource Management Minor
Human Rights Minor
Human Security and Geospatial Intelligence Minor
Immersive Media Minor
Individuals, Societies and Aging Minor
Innovation: The Digital Entrepreneur Minor
International Health, De

Economic Developmental Programming (MA)
Economics (MA)
Educational Counseling (MEd)
Electrical Engineering (Computer Networks) (MS)
Electrical Engineering (Electric Power) (MS)
Electrical Engineering (MS)
Electrical Engineering (Multimedia and Creative Technologies) (MS)
Electrical Engineering (VLSI Design) (MS)
Electrical Engineering (Wireless Health Technology) (MS)
Electrical Engineering (Wireless Networks) (MS)
Engineering Management (MS)
English (MA)
Entrepreneurship and Innovation (MS)
Environmental Engineering (MS)
Environmental Risk Analysis (MS)
Environmental Studies (MA)
Experimental and Molecular Pathology (MS)
Finance (MS)
Financial Engineering (MS)
Fine Arts (MFA)
Geographic Information Science and Technology (MS)
Geological Sciences (MS)
Geriatric Dentistry (MS)
Gerontology (MA)
Gerontology (MS)
Global Communication (MA)
Global Medicine (MS)
Global Supply Chain Management (On‑Campus/Residential) (MS)
Global Supply Chain Management for Executives (MS) (Online)
Green Techno

Geriatric Dentistry Graduate Certificate
Gifted Education Certificate
Health Management and Policy Programs Certificate
Heritage Conservation Certificate
History of Collecting and Display Certificate
Homeland Security and Public Policy Certificate
International Policy and Planning Certificate
Landscape Architecture Certificate
Library and Information Management Graduate Certificate
Management Studies Graduate Certificate
Marketing Graduate Certificate
Nonprofit Management and Policy Certificate
Optimization and Supply Chain Management Graduate Certificate
Oral Pathology and Radiology Certificate
Orofacial Pain Certificate
Pain Medicine Certificate
Political Management Certificate
Public Financial Management Certificate
Public Management Certificate
Public Policy Certificate
Real Estate Development Certificate
Religious Studies Certificate
School Counseling Certificate
Social Innovation Certificate
Social Justice Certificate
Special Education Certificate
STEM Education Certificate
Strat

In [234]:
with open('Catalogue.json', 'w') as f:
    json.dump(program_json, f, indent=4)